In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pymc3 as pm
import theano

np.random.seed(42)
%matplotlib inline

import os
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
from model.gr4j_np import GR4J

ModuleNotFoundError: No module named 'model'

In [ ]:
data_location = os.path.join('/data')

In [ ]:
# Calibration period
calib_start_year = 1990
calib_end_year = 2000

# Holdout period
holdout_start_year = 2000
holdout_end_year = 2005

In [ ]:
# Load the data
maulescreek_rain_et = pd.read_csv(
    os.path.join(
        data_location,
        'SILO(-30.5,150.1)',
        'FAO56_Penman_Monteith_1989.csv'
    )
)
maulescreek_streamflow = pd.read_csv(
    os.path.join(
        data_location,
        'water_level',
        'bom_gauge_daily_mean_419051.csv'
    )
)

# fix dates 
maulescreek_rain_et['Date'] = pd.to_datetime(maulescreek_rain_et['Date2'],format="%d-%m-%Y")
maulescreek_streamflow['Date'] = pd.to_datetime(maulescreek_streamflow['Timestamp'],utc=True, format='%Y-%m-%d')
maulescreek_streamflow['Date'] = maulescreek_streamflow['Date'].dt.date
maulescreek_streamflow['Date'] = pd.to_datetime(maulescreek_streamflow['Date'])

In [ ]:
# 
merged_data = pd.merge(
    maulescreek_rain_et,
    maulescreek_streamflow,
    left_on='Date',
    right_on='Date'
)

# Slicing the data into calibration and holdout
calibration_data = merged_data.loc[((merged_data['Date']).dt.year >= calib_start_year) & ((merged_data['Date']).dt.year < calib_end_year)].reset_index(drop=True)

holdout_data = merged_data.loc[((merged_data['Date']).dt.year >= holdout_start_year) & ((merged_data['Date']).dt.year < holdout_end_year)].reset_index(drop=True)

In [ ]:
# Tune GR4J parameters
prcp_calib = calibration_data.loc[:, 'Rain'].values
etp_calib = calibration_data.loc[:, 'FAO56'].values
flow_calib = calibration_data.loc[:, 'Value'].values
date_calib = calibration_data.loc[:, 'Date'].values

In [ ]:
eps = 0.1

In [ ]:
gr4j = GR4J()

with pm.Model() as model:

    x1 = pm.Normal('x1', 400, 100)
    x2 = pm.Normal('x2', 0, 16)
    x3 = pm.Normal('x3', 100, 64)
    x4 = pm.InverseGamma('x4', alpha=1, beta=1)
    s = pm.Exponential('error', 1)
    
    gr4j.update_params(x1, x2, x3, x4)

    q_hat = gr4j.run(prcp_calib, etp_calib)[0]

    # predictions
    obs = pm.Normal('observation', q_hat, s, observed=flow_calib)

    # use MCMC to sample
    trace = pm.sample(step=pm.Metropolis([x1, x2, x3, x4, s]))



TypeError: 'TensorVariable' object cannot be interpreted as an integer